# Домашнее задание №1

На семинаре мы проходили метрики, необходимо реализовать некоторые из них

In [8]:
import numpy as np
import pandas as pd
import math

## Задание 1. Реализовать метрики Recall@k и  Money Recall@k

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

$$\Large Recall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}}{|Rel_i|}$$

$\Large |Rel_i|$ -- количество релевантных товаров для пользователя $i$

$$\Large MoneyRecall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}\cdot Price(j)}{\sum_{s\in Rel_i}Price(s)}$$


In [18]:
def recall_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    recommended_list = recommended_list[:k]
    flags = np.isin(recommended_list,bought_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    prices_bought = np.array(prices_bought)
    
    flags = np.isin(recommended_list,bought_list)
    money_recall = (flags*prices_recommended).sum() / (bought_list*prices_bought).sum()
    
    return money_recall

## Задание 2. Реализовать метрику MRR@k

Mean Reciprocal Rank

- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $\Large rank_j$
- Посчитать reciprocal rank = $\Large\frac{1}{rank_j}$

$$\Large  ReciprocalRank(i)@k=\frac {1}{\min\limits_{j\in Rel(i)} rank_j}$$

In [23]:
def reciprocal_rank_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    recommended_list = recommended_list[:k]
    rank = 1
    for i in range(len(recommended_list)):
        if np.isin(recommended_list[i],bought_list):
            rank = i+1
            break
        else:
            continue  
    rr_at_k = 1/rank

    return rr_at_k

## Задание 3*. Реализовать метрику nDCG@k
Normalized discounted cumulative gain. Эту метрику реализовать будет немного сложнее.

$$\Large DCG@K(i) = \sum_{j=1}^{K}\frac{\mathbb{1}_{r_{ij}}}{\log_2 (j+1)}$$


$\Large \mathbb{1}_{r_{ij}}$ -- индикаторная функция показывает что пользователь $i$ провзаимодействовал с продуктом $j$

Для подсчета $nDCG$ нам необходимо найти максимально возможный $DCG$ для пользователя $i$  и рекомендаций длины $K$.
Максимальный $DCG$ достигается когда мы порекомендовали максимально возможное количество релевантных продуктов и все они в начале списка рекомендаций.

$$\Large IDCG@K(i) = max(DCG@K(i)) = \sum_{j=1}^{K}\frac{\mathbb{1}_{j\le|Rel_i|}}{\log_2 (j+1)}$$

$$\Large nDCG@K(i) = \frac {DCG@K(i)}{IDCG@K(i)}$$

$\Large |Rel_i|$ -- количество релевантных продуктов для пользователя $i$



In [29]:
def ndcg_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    recommended_list = recommended_list[:k]
    dcg_at_k = 0
    flags = np.isin(recommended_list,bought_list)
    for i in range(len(flags)):
        dcg_at_k += flags[i]/math.log2(i+2)
   
    idcg_at_k = 0
    for i in range(len(flags)):
        if flags[i]:
            idcg_at_k = 1/math.log2(i+2)
            break
        else:
            continue
        
    ndcg_at_k = dcg_at_k/idcg_at_k
    
    return ndcg_at_k
    

Проверка:

In [15]:
recommended_list = [146, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43]
bought_list = [521, 27, 143, 991]
prices_recommended = [100, 500, 200, 250, 175, 500, 25, 300, 244, 155]
prices_bought = [250, 250, 1000, 57]

In [25]:
recall_at_k(recommended_list, bought_list)

0.5

In [19]:
money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought)

0.0012630502812887273

In [24]:
reciprocal_rank_at_k(recommended_list, bought_list)

0.25

In [30]:
ndcg_at_k(recommended_list, bought_list)

1.8982444017039275